# Preparations

## Imports and configuration

In [ ]:
import os
import sys
sys.path.append('..')
import pypmj as jpy
import numpy as np

# Simulation

## Project set-up

In [ ]:
project = jpy.JCMProject('scattering/mie/mie2D_post_processes')

## Prepare the simulation set

In [ ]:
mie_keys = {'constants' :{},
            'parameters': {},
            'geometry': {'radius':np.linspace(0.3, 0.5, 40)}}

Now, the `SimulationSet` can be initialized.

In [ ]:
simuset = jpy.SimulationSet(project, mie_keys, duplicate_path_levels=0,
                            storage_folder='mie2D_test_pp',
                            storage_base=os.path.abspath('tmp_storage_folder'))

In [ ]:
simuset.make_simulation_schedule()

In [ ]:
simuset.use_only_resources('localhost')
simuset.resource_manager.resources.set_m_n_for_all(4,1)

## Running a single simulation with additional post process(es)

In [ ]:
pp_file = os.path.join(project.working_dir, 'flux_computation.jcmp')
results, logs = simuset.solve_single_simulation(0, run_post_process_files=pp_file)

In [ ]:
print simuset.simulations[0].logs['Out']

## Using a processing function

In [ ]:
def read_scs(pp):
    results = {} #must be a dict
    results['SCS'] = pp[0]['ElectromagneticFieldEnergyFlux'][0][0].real
    return results

In [ ]:
simuset.simulations[0].process_results(processing_func=read_scs, overwrite=True)

In [ ]:
'SCS' in simuset.simulations[0]._results_dict

## Running all simulations with (an) additional post process(es)

In [ ]:
simuset.run(N=10, processing_func=read_scs, run_post_process_files=pp_file)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context('notebook')

data = simuset.get_store_data().sort_values(by='radius')
data.plot(x='radius', y='SCS', title='Results of the simulation')
plt.show()